In [ ]:
import collections
import json
import pathlib

import matplotlib.ticker
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

from hubbardml import datasets
from hubbardml import keys
from hubbardml import plots

import olivines

In [ ]:
# The set of experiments to consider 
EXPERIMENTS = tuple(
    map(
        pathlib.Path,
        ("outputs/olivines/batch_size=8,max_epochs=6000/2024-04-08_14-39-31",
         "outputs/olivines/batch_size=8,max_epochs=6000/2024-05-06_17-39-53",
         "outputs/olivines/batch_size=8,max_epochs=6000/2024-05-07_11-53-29",)
    )
)

EXPERIMENT_DIR = EXPERIMENTS[0]

DATASET = "dataset.json"
RESULTS_FILE = "hubbard_u_olivines.json"
OCC_STATE = "occ_state"

In [ ]:
all_results = tuple(map(lambda path: pd.read_json(path / RESULTS_FILE), EXPERIMENTS))

In [ ]:
uv_data = pd.read_json(EXPERIMENT_DIR / DATASET)
labels = {
    'LiFePO4': 'Li$_{x}$FePO$_4$',
    'LiMnPO4': 'Li$_{x}$MnPO$_4$',
    "LiFe0.5Mn0.5PO4": "Li$_{x}$Fe$_{0.5}$Mn$_{0.5}$PO$_4$",
}
uv_data

In [ ]:
uv_data[uv_data[keys.PARAM_TYPE] == keys.PARAM_V].groupby(keys.MATERIAL).value_counts([olivines.Keys.OCCUPATION])

In [ ]:
def create_plots(df: pd.DataFrame, logy=False, scale=1.0, include_training=False, experiments: pd.DataFrame = None):
    comparison_plots = {}

    if include_training and olivines.Keys.TRAIN_RMSE in df:
        cols = [olivines.Keys.TRAIN_RMSE, olivines.Keys.MODEL_RMSE]
        series_labels = 'ML (training)', 'ML (validation)'
        colors = plots.train_validate_colours[keys.TRAIN], plots.train_validate_colours[keys.VALIDATE]
    else:
        cols = [olivines.Keys.MODEL_RMSE]
        series_labels = ('ML',)
        colors = (plots.train_validate_colours[keys.VALIDATE],)

    for idx, material in enumerate(df[olivines.Keys.MATERIAL].unique()):
        frame = df[df[olivines.Keys.MATERIAL] == material]

        if experiments is None:
            ref_rmse = frame.iloc[0][olivines.Keys.REF_RMSE]  # They are all the same
            ref_rmse_std = None
        else:
            # Calculate an average over multiple experiments
            rmses = []
            for entry in experiments:
                sub = entry[entry[olivines.Keys.MATERIAL] == material]
                rmses.append(sub.iloc[0][olivines.Keys.REF_RMSE])
            ref_rmse = np.mean(rmses)
            ref_rmse_std = np.std(rmses)

        grouped = frame.groupby(olivines.Keys.NUM_OCCUPATIONS)[cols]
        mins = grouped.min()
        maxs = grouped.max()
        means = grouped.mean()

        # print(material, ref_rmse, means)

        errors = [[means[c] - mins[c], maxs[c] - means[c]] for c in cols]

        fig = plt.figure(figsize=(6 * scale, 4 * scale))
        ax = fig.gca()

        ax = means.plot(
            ax=ax,
            # colormap=plots.colourmap,
            color=colors,  # plots.train_validate_colours.values(),
            yerr=errors,
            fmt='o--',
            logy=logy,
            solid_capstyle='projecting',
            capsize=5,
            capthick=2,
            # title=labels[material],
            # linewidth=3,
            # alpha=0.5,
            ylabel='RMSE (eV)',
            xlabel='$N_\mathrm{c}$ ',
            legend=False,
        );

        # Plot the baseline RMSE
        ref_range = np.array([df[olivines.Keys.NUM_OCCUPATIONS].min(), df[olivines.Keys.NUM_OCCUPATIONS].max()])
        ref_rmse = np.array([ref_rmse, ref_rmse])
        ax.plot(
            ref_range,
            ref_rmse,
            c=plots.train_validate_colours[keys.REFERENCE]
        )
        if idx == 2:
            ax.legend((f"Reference",) + series_labels)

        if ref_rmse_std is not None:
            ax.fill_between(ref_range, ref_rmse - ref_rmse_std, ref_rmse + ref_rmse_std, alpha=0.1)

        # Plot the individual points
        for oxidation in "2+", "3+":
            for _idx, row in frame[frame["oxidation"] == oxidation].iterrows():
                ax.annotate(
                    oxidation,
                    xy=(row[olivines.Keys.NUM_OCCUPATIONS], row[olivines.Keys.MODEL_RMSE]),
                    xytext=(row[olivines.Keys.NUM_OCCUPATIONS] + 20, row[olivines.Keys.MODEL_RMSE] - 12),
                    fontsize=9,
                    arrowprops=dict(facecolor="black", arrowstyle="-"),
                    textcoords='offset pixels',
                    # ha='left',
                    # va='bottom',
                )
                # ax.text(
                #     row[olivines.Keys.NUM_OCCUPATIONS] + 0.07,
                #     row[olivines.Keys.MODEL_RMSE],
                #     oxidation,
                #     fontsize=11,
                #     ha='left',
                #     va='center',
                # )

        ax.scatter(
            frame[olivines.Keys.NUM_OCCUPATIONS],
            frame[olivines.Keys.MODEL_RMSE],
            c=plots.train_validate_colours[keys.VALIDATE],
            s=12,
            alpha=0.5,
        )

        ax.yaxis.set_major_formatter(plt.FormatStrFormatter('%.2f'))
        # ax.legend((f"Ref {ref_rmse * 1000:.0f} meV",) + series_labels)

        ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
        # ax.set_ylim([0, 1.2])

        comparison_plots[material] = ax

    return comparison_plots

# Olivines analysis

In [ ]:
results_file = EXPERIMENT_DIR / RESULTS_FILE
with open(results_file, 'r') as file:
    hubbard_u_olivines = pd.DataFrame(json.load(file))

In [ ]:
def to_oxidation(occ):
    if occ == ['0.00']:
        return "3+"
    if occ == ['1.00']:
        return "2+"

    return "2/3+"


hubbard_u_olivines["oxidation"] = hubbard_u_olivines["occupation"].apply(to_oxidation)
hubbard_u_olivines

In [ ]:
save_to = EXPERIMENT_DIR / "plots"
save_to.mkdir(exist_ok=True)
save_to

In [ ]:
for material, ax in create_plots(hubbard_u_olivines, scale=0.7, experiments=all_results).items():
    path = save_to / f"{material}_U_occs_comparison.pdf"
    ax.get_figure().savefig(path, bbox_inches='tight')
    print(path.absolute())


In [ ]:
uv_data[keys.SPECIES] = uv_data[keys.SPECIES].map(frozenset)

In [ ]:
fig = plots.plot_param_histogram(
    uv_data,
    bins=30,
    density=False
)
fig.set_size_inches(5, 2)
path = EXPERIMENT_DIR / "plots" / "param_histogram.pdf"
fig.savefig(path, bbox_inches='tight')
path.absolute()

In [ ]:
results_file = 'hubbard_v_olivines.json'
with open(results_file, 'r') as file:
    hubbard_v_olivines = pd.DataFrame(json.load(file))

for material, ax in create_plots(hubbard_v_olivines, logy=True, scale=0.7).items():
    ax.get_figure().savefig(save_to / "f{material}_V_occs_comparison.pdf", bbox_inches='tight')

save_to

In [ ]:
for material in olivines.MATERIALS:
    print(material)
    subset = uv_data[uv_data[keys.DIR].str.contains(material)]
    elements = subset[keys.ATOM_1_ELEMENT].unique()

    for element in elements:
        elementsubset = subset[subset[keys.ATOM_1_ELEMENT] == element]
        minval = elementsubset[keys.PARAM_OUT].min()
        maxval = elementsubset[keys.PARAM_OUT].max()
        stdev = elementsubset[keys.PARAM_OUT].std()

        meanval = elementsubset[keys.PARAM_OUT].mean()
        print(len(elementsubset), element, minval, maxval, meanval, stdev)


In [ ]:
olivines.MATERIALS

In [ ]:
ds = pd.read_json(EXPERIMENT_DIR / "LiMnPO4_2_0.25-0.50" / DATASET)

In [ ]:
def relative_error(row):
    return np.abs((row[keys.PARAM_OUT_PREDICTED] - row[keys.PARAM_OUT]) / row[keys.PARAM_OUT])


ds[ds[keys.TRAINING_LABEL] == keys.VALIDATE].apply(relative_error, axis=1).hist()